In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
%matplotlib inline

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
data.test.cls = np.argmax(data.test.labels, axis=1)
data.train.cls = np.argmax(data.train.labels, axis=1)

In [4]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x2": np.array(data.train.images)},
    y=np.array(data.train.cls),
    num_epochs=None,
    batch_size=1024,
    shuffle=True)

In [5]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x2": np.array(data.test.images)},
    y=np.array(data.test.cls),
    num_epochs=1,
    shuffle=False)

In [6]:
feature_x = tf.feature_column.numeric_column("x2", shape=(784))

In [7]:
feature_columns = [feature_x]
feature_columns

[_NumericColumn(key='x2', shape=(784,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [8]:
def model_fn(features, labels, mode, params):
  x = features["x2"]
  net = tf.layers.dense(inputs=x, name='h1',units=512, activation=tf.nn.relu)
  net2 = tf.layers.dense(inputs=net, name='h2',units=256, activation=tf.nn.relu)
  net3 = tf.layers.dense(inputs=net2, name='h3',units=128, activation=tf.nn.relu)
  net4 = tf.layers.dense(inputs=net3, name='softmax',units=10,activation=tf.nn.softmax)
  
  y_pred_cls = tf.argmax(net4, axis=1)
  
  if mode == tf.estimator.ModeKeys.PREDICT:
    spec = tf.estimator.EstimatorSpec(mode=mode,predictions=y_pred_cls)
    
  else:
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,logits = net4)
    loss = tf.reduce_mean(cross_entropy)
    optimizer = tf.train.ProximalAdagradOptimizer(learning_rate=params["learning_rate"],l1_regularization_strength=0.001)
    train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    accuracy = tf.metrics.accuracy(labels, y_pred_cls)
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('train_accuracy', accuracy[1])
    
    tf.summary.histogram("hidden1",net)
    tf.summary.histogram("hidden2",net2)
    tf.summary.histogram("hidden3",net3)
    
    spec = tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op,eval_metric_ops=metrics)
    
  return spec

In [9]:
params = {"learning_rate": 0.1}
model = tf.estimator.Estimator(model_fn=model_fn,
                               params=params,model_dir='/content/datalab/docs/log10/')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc90a5a0da0>, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/content/datalab/docs/log10/', '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_session_config': None, '_tf_random_seed': None, '_num_ps_replicas': 0, '_save_checkpoints_steps': None, '_task_type': 'worker', '_master': '', '_is_chief': True, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_service': None, '_task_id': 0}


In [10]:
model.train(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /content/datalab/docs/log10/model.ckpt-1000
INFO:tensorflow:Saving checkpoints for 1001 into /content/datalab/docs/log10/model.ckpt.
INFO:tensorflow:loss = 1.7860029, step = 1001
INFO:tensorflow:global_step/sec: 6.16214
INFO:tensorflow:loss = 1.7354205, step = 1101 (16.240 sec)
INFO:tensorflow:global_step/sec: 6.01177
INFO:tensorflow:loss = 1.7407854, step = 1201 (16.628 sec)
INFO:tensorflow:global_step/sec: 6.17282
INFO:tensorflow:loss = 1.7134619, step = 1301 (16.200 sec)
INFO:tensorflow:global_step/sec: 6.10859
INFO:tensorflow:loss = 1.6671824, step = 1401 (16.371 sec)
INFO:tensorflow:global_step/sec: 6.0716
INFO:tensorflow:loss = 1.7077456, step = 1501 (16.470 sec)
INFO:tensorflow:global_step/sec: 6.13853
INFO:tensorflow:loss = 1.7067506, step = 1601 (16.290 sec)
INFO:tensorflow:global_step/sec: 6.16104
INFO:tensorflow:loss = 1.7213932, step = 1701 (16.232 sec)
INFO:tensorflow:global_step/sec: 6.1

In [50]:
from google.datalab.ml import TensorBoard as tb

In [51]:
tb.start('/content/datalab/docs/log10/')

TensorBoard was started successfully with pid 7583. Click here to access it.

7583